In [39]:
#La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados y que no están logueados (⭐)

In [40]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,499 kB]
Get

In [41]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [44]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [53]:
rdd.take(5)

[Row(title='Wikipedia:Artículos solicitados', id='5', namespace='4', revision_id='132533307', parent_revision='132516631', revision_timestamp='2021-01-19T18:40:34Z', revisor_username='Metrónomo', revisor_id='1708233', revisor_ip=None, revisor_comment='Así esta mejor'),
 Row(title='Andorra', id='7', namespace='0', revision_id='138035057', parent_revision='138027475', revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='Añadir contenido'),
 Row(title='Argentina', id='10', namespace='0', revision_id='137966826', parent_revision='137825165', revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id='4642283', revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: «simbolos»'),
 Row(title='Geografía de Andorra', id='15', namespace='0', revision_id='137986788', parent_revision='133666383', revision_timestamp='2021-08-29T17:16:00Z',

Cuento cuantas revisiones fueron hechas por usuarios registrados

In [47]:
tot_usuarios_registrados = rdd.filter(lambda x: x.revisor_id).count()
tot_usuarios_registrados

3938877

Obtengo cuantas revisiones fueron hechas por usuarios NO registrados. Usando el total y las que hicieron los registrados

In [48]:
tot_usuarios_no_registrados = ( rdd.count() - tot_usuarios_registrados ) # --> rdd.filter(lambda x: x.revisor_id == None).count()
tot_usuarios_no_registrados

193756

Cuento cuantas revisiones hechas por usuarios registrados no tienen comentario

In [49]:
registrados_sin_coment = rdd.filter(lambda x: x.revisor_id and not x.revisor_comment).count()
registrados_sin_coment

392200

Hago lo mismo para usuarios NO registrados

In [50]:
no_registrados_sin_coment = rdd.filter(lambda x: not x.revisor_id and not x.revisor_comment).count()
no_registrados_sin_coment

77544

Porcentaje para usuarios registrados

In [51]:
print("La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados: {}%".format((registrados_sin_coment*100) / tot_usuarios_registrados) )

La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados: 9.957152762068986%


Porcentaje para usuarios no registrados

In [52]:
print("La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que No están logueados: {}%".format((no_registrados_sin_coment*100)/tot_usuarios_no_registrados) )

La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que No están logueados: 40.021470302855136%
